In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.optim.lr_scheduler import LinearLR
import torch.nn as nn
import torch
import gc
import os
import imageio
import yaml
from torch.optim import Adam
from datetime import datetime
from PIL import Image
import requests
import nbconvert
from google.colab import drive

In [ ]:
# File upload (Make sure 'Image-01.png' is uploaded to /content/)
image_file_name = "Image-01.png"
image_path = os.path.join("/content/", image_file_name) # Basic upload path

# Open image
try:
    original_image = Image.open(image_path)
    print(f"'{image_file_name}' Successfully opened.")
except FileNotFoundError:
    print(f"Cannot find image. Please upload '{image_file_name}' to /content/")
except Image.UnidentifiedImageError:
    print(f"{image_path} is not a valid image file.")
except Exception as e:
    print(f"Undefined error: {e}")

In [ ]:
# [CODE BLOCK 1: Function Definition]

def generate_2D_gaussian_splatting(
    kernel_size, scale, rotation, coords, colours, image_size=(256, 256, 3),
    device="cpu", filter_type=None, debug_print=False):
    """
    Generates a 2D image by splatting multiple Gaussians.

    Args:
        kernel_size (int): Size of the square kernel for each Gaussian.
        scale (torch.Tensor): Tensor of shape (N, 2) representing the x and y scales of N Gaussians.
        rotation (torch.Tensor): Tensor of shape (N) representing the rotation angle (radians) of N Gaussians.
        coords (torch.Tensor): Tensor of shape (N, 2) representing the normalized ([-1, 1]) center coordinates (x, y) of N Gaussians.
        colours (torch.Tensor): Tensor of shape (N, 3) representing the RGB (with alpha pre-multiplied) colors of N Gaussians.
        image_size (tuple, optional): Target image dimensions (height, width, channels). Defaults to (256, 256, 3).
        device (str, optional): Device to run computations on ('cpu' or 'cuda'). Defaults to "cpu".
        filter_type (str, optional): Type of anti-aliasing filter.
                                     Can be 'low_pass', 'ewa', or None. Defaults to None.
        debug_print (bool, optional): If True, prints covariance matrix changes for debugging.
                                      Defaults to False.
    """

    batch_size = colours.shape[0]

    # Ensure scale and rotation have the correct shape
    scale = scale.view(batch_size, 2)
    rotation = rotation.view(batch_size)

    # Compute the components of the covariance matrix
    cos_rot = torch.cos(rotation)
    sin_rot = torch.sin(rotation)

    R = torch.stack([
        torch.stack([cos_rot, -sin_rot], dim=-1),
        torch.stack([sin_rot, cos_rot], dim=-1)
    ], dim=-2)

    S = torch.diag_embed(scale)

    # Calculate the original covariance matrix: Σ_splat = R @ S @ S @ R^T
    covariance = R @ S @ S @ R.transpose(-1, -2)

    # For debugging: store the original covariance before filtering
    if debug_print:
        covariance_before = covariance.clone()

    # --- Apply anti-aliasing filters ---
    # This section modifies the 'covariance' matrix based on the chosen filter_type
    if filter_type == 'low_pass':
    # Implement Low-Pass Filter:
    # Goal: If a Gaussian is too small (causing aliasing), make it slightly larger (blurrier).
    # 1. Determine a 'threshold' for the Gaussian's size (e.g., related to pixel dimensions).
    # 2. Calculate the eigenvalues of the 'covariance' matrix to determine its spread.
    # 3. Identify Gaussians whose spread (eigenvalues) falls below the threshold.
    # 4. For these identified Gaussians, add a small isotropic (uniform) blur amount to their 'covariance' matrix (e.g., add 'blur_amount_square * torch.eye(2)' to it).

        pass # Placeholder, remove this line when implementing

    elif filter_type == 'ewa':
   # Implement EWA Filter:
   # Goal: Convolve the Gaussian with a pixel filter Gaussian.
   # This conceptually means the final rendered Gaussian's covariance is the sum
   # of the original Gaussian's covariance and the pixel filter's covariance.
   # 1. Define a 'pixel_radius' (e.g., 0.5 for a standard pixel).
   # 2. Create the covariance matrix for the pixel filter (Sigma_pixel) based on 'pixel_radius'.
   #    It should be an isotropic (circular) blur.
   # 3. Add Sigma_pixel to the original 'covariance' matrix. This applies to all Gaussians.

        pass # Placeholder, remove this line when implementing

    # For debugging: print comparison of covariance matrices and eigenvalues
    if debug_print:
        scales_det = scale[:, 0] * scale[:, 1]
        target_idx = torch.argmin(scales_det)

        print(f"--- Debugging Anti-Aliasing Filter: '{filter_type if filter_type else 'None'}' ---\n")
        print(f"Target Gaussian Index (smallest by scale): {target_idx.item()}")
        print("Covariance Before Filtering (Smallest Gaussian):\n", covariance_before[target_idx].detach().cpu().numpy())
        print("Covariance After Filtering (Smallest Gaussian):\n", covariance[target_idx].detach().cpu().numpy())

        eig_before = torch.linalg.eigvalsh(covariance_before[target_idx])
        eig_after = torch.linalg.eigvalsh(covariance[target_idx])
        print("Eigenvalues Before Filtering:\n", eig_before.detach().cpu().numpy())
        print("Eigenvalues After Filtering:\n", eig_after.detach().cpu().numpy())
        print("-" * 60 + "\n")

    # Compute inverse covariance (will use the potentially filtered covariance)
    inv_covariance = torch.inverse(covariance)

    # Create the kernel grid
    x = torch.linspace(-5, 5, kernel_size, device=device)
    y = torch.linspace(-5, 5, kernel_size, device=device)
    xx, yy = torch.meshgrid(x, y, indexing='ij')
    xy = torch.stack([xx, yy], dim=-1).unsqueeze(0).expand(batch_size, -1, -1, -1)

    # Calculate the exponent part of the Gaussian PDF
    z = torch.einsum('bxyi,bij,bxyj->bxy', xy, -0.5 * inv_covariance, xy)

    # Calculate the full Gaussian PDF (integral is 1). This is the correct base intensity.
    kernel = torch.exp(z) / (2 * torch.tensor(np.pi, device=device) * torch.sqrt(torch.det(covariance))).view(batch_size, 1, 1)

    # Use the 'kernel' (PDF values) directly.
    # The 'colours' parameter already includes alpha from the caller (colours_with_alpha).
    kernel_rgb_base = kernel.unsqueeze(1).expand(-1, 3, -1, -1)

    # Add padding to match target image size
    pad_h = image_size[0] - kernel_size
    pad_w = image_size[1] - kernel_size

    if pad_h < 0 or pad_w < 0:
        raise ValueError("Kernel size should be smaller or equal to the image size.")

    # Padding dimensions: (left, right, top, bottom)
    padding = (pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2)
    kernel_rgb_padded = F.pad(kernel_rgb_base, padding, "constant", 0)

    # Create affine grid for translating the kernel to Gaussian's coordinates
    b, c, h, w = kernel_rgb_padded.shape
    theta = torch.zeros(b, 2, 3, dtype=torch.float32, device=device)
    theta[:, 0, 0] = 1.0
    theta[:, 1, 1] = 1.0
    theta[:, :, 2] = coords

    grid = F.affine_grid(theta, size=(b, c, h, w), align_corners=True)
    kernel_rgb_padded_translated = F.grid_sample(kernel_rgb_padded, grid, align_corners=True)

    # Apply colors (which already include alpha from the input 'colours') and sum the layers
    # rgb_values_reshaped holds (RGB * Alpha) from the input 'colours' parameter
    rgb_values_reshaped = colours.unsqueeze(-1).unsqueeze(-1)
    final_image_layers = rgb_values_reshaped * kernel_rgb_padded_translated
    final_image = final_image_layers.sum(dim=0)

    # Clamp values to [0, 1] and permute dimensions for imshow (H, W, C)
    final_image = torch.clamp(final_image, 0, 1)
    final_image = final_image.permute(1, 2, 0)

    return final_image

In [ ]:
def create_window(window_size, channel):
    def gaussian(window_size, sigma):
        gauss = torch.exp(torch.tensor([-(x - window_size//2)**2/float(2*sigma**2) for x in range(window_size)]))
        return gauss/gauss.sum()

    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = torch.autograd.Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())

    return window



def ssim(img1, img2, window_size=11, size_average=True):

    (_, _, channel) = img1.size()

    img1 = img1.unsqueeze(0).permute(0, 3, 1, 2)
    img2 = img2.unsqueeze(0).permute(0, 3, 1, 2)


    # Parameters for SSIM
    C1 = 0.01**2
    C2 = 0.03**2

    window = create_window(window_size, channel)

    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)

    mu1 = F.conv2d(img1, window, padding=window_size//2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size//2, groups=channel)
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1*img1, window, padding=window_size//2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding=window_size//2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding=window_size//2, groups=channel) - mu1_mu2

    SSIM_numerator = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))
    SSIM_denominator = ((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))
    SSIM = SSIM_numerator / SSIM_denominator

    return torch.clamp((1 - SSIM) / 2, 0, 1)

def d_ssim_loss(img1, img2, window_size=11, size_average=True):
    return ssim(img1, img2, window_size, size_average).mean()

# Combined Loss
def combined_loss(pred, target, lambda_param=0.5):
    l1loss = nn.L1Loss()
    return (1 - lambda_param) * l1loss(pred, target) + lambda_param * d_ssim_loss(pred, target)

In [ ]:
# Read the config.yml file
with open('config.yml', 'r') as config_file:
    config = yaml.safe_load(config_file)

# Extract values from the loaded config
KERNEL_SIZE = config["KERNEL_SIZE"]
image_size = tuple(config["image_size"])
primary_samples = config["primary_samples"]
backup_samples = config["backup_samples"]
num_epochs = config["num_epochs"]
densification_interval = config["densification_interval"]
learning_rate = config["learning_rate"]
image_file_name = config["image_file_name"]
display_interval = config["display_interval"]
grad_threshold = config["gradient_threshold"]
gauss_threshold = config["gaussian_threshold"]
display_loss = config["display_loss"]

In [ ]:
def give_required_data(input_coords, image_size):

  # normalising pixel coordinates [-1,1]
  coords = torch.tensor(input_coords / [image_size[0],image_size[1]], device=device).float()
  center_coords_normalized = torch.tensor([0.5, 0.5], device=device).float()
  coords = (center_coords_normalized - coords) * 2.0

  # Fetching the colour of the pixels in each coordinates
  colour_values = [image_array[coord[1], coord[0]] for coord in input_coords]
  colour_values_np = np.array(colour_values)
  colour_values_tensor =  torch.tensor(colour_values_np, device=device).float()

  return colour_values_tensor, coords

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_samples = primary_samples + backup_samples

PADDING = KERNEL_SIZE // 2
image_path = image_file_name
original_image = Image.open(image_path)
original_image = original_image.resize((image_size[0],image_size[0]))
original_image = original_image.convert('RGB')
original_array = np.array(original_image)
original_array = original_array / 255.0
width, height, _ = original_array.shape

image_array = original_array
target_tensor = torch.tensor(image_array, dtype=torch.float32, device=device)
coords = np.random.randint(0, [width, height], size=(num_samples, 2))
random_pixel_means = torch.tensor(coords, device=device)
pixels = [image_array[coord[1], coord[0]] for coord in coords]
pixels_np = np.array(pixels)
random_pixels =  torch.tensor(pixels_np, device=device)

colour_values, pixel_coords = give_required_data(coords, image_size)

colour_values = torch.logit(colour_values)
pixel_coords = torch.atanh(pixel_coords)

scale_values = torch.logit(torch.rand(num_samples, 2, device=device))
rotation_values = torch.atanh(2 * torch.rand(num_samples, 1, device=device) - 1)
alpha_values = torch.logit(torch.rand(num_samples, 1, device=device))
W_values = torch.cat([scale_values, rotation_values, alpha_values, colour_values, pixel_coords], dim=1)


In [ ]:
starting_size = primary_samples
left_over_size = backup_samples
persistent_mask = torch.cat([torch.ones(starting_size, dtype=bool),torch.zeros(left_over_size, dtype=bool)], dim=0)
current_marker = starting_size

In [ ]:
# Get current date and time as string
now = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")

# Create a directory with the current date and time as its name
directory = f"{now}"
os.makedirs(directory, exist_ok=True)

In [ ]:
W = nn.Parameter(W_values)
optimizer = Adam([W], lr=learning_rate)
scheduler = LinearLR(optimizer, start_factor=1.0, end_factor=1e-8, total_iters=num_epochs)
loss_history = []

In [ ]:
# [CODE BLOCK for Training Loop]

for epoch in range(num_epochs):

    # Find indices to remove and update the persistent mask (Pruning logic)
    if epoch % (densification_interval + 1) == 0 and epoch > 0:
        indices_to_remove = (torch.sigmoid(W[:, 3]) < 0.01).nonzero(as_tuple=True)[0]

        if len(indices_to_remove) > 0:
          print(f"Number of pruned points: {len(indices_to_remove)}")

        persistent_mask[indices_to_remove] = False

        # Zero-out parameters and their gradients for pruned Gaussians
        W.data[~persistent_mask] = 0.0


    gc.collect()
    torch.cuda.empty_cache()

    # Select active Gaussians based on persistent mask
    output = W[persistent_mask]

    batch_size = output.shape[0]

    # Decode Gaussian parameters from the output tensor
    scale = torch.sigmoid(output[:, 0:2])
    rotation = np.pi / 2 * torch.tanh(output[:, 2]) # Scale rotation to [-pi/2, pi/2]
    alpha = torch.sigmoid(output[:, 3])
    colours = torch.sigmoid(output[:, 4:7])
    pixel_coords = torch.tanh(output[:, 7:9]) # Coords are in [-1, 1] range

    colours_with_alpha  = colours * alpha.view(batch_size, 1)

    # --- Main Training Loop Rendering Call ---
    # The model is trained based on a BASELINE (no filter) output.
    # debug_print is OFF by default here for performance.
    g_tensor_batch = generate_2D_gaussian_splatting(
        KERNEL_SIZE, scale, rotation, pixel_coords, colours_with_alpha,
        image_size, device=device,
        filter_type=None, # <--- Model is trained with NO FILTER (Baseline)
        debug_print=False  # Keep debug_print off during main training
    )
    # Calculate loss against the target image
    loss = combined_loss(g_tensor_batch, target_tensor, lambda_param=0.2)

    optimizer.zero_grad() # Clear gradients before backpropagation
    loss.backward()       # Perform backpropagation to compute gradients

    # Apply zeroing out of gradients for pruned Gaussians
    if persistent_mask is not None:
        W.grad.data[~persistent_mask] = 0.0

    # Densification logic (Splitting and Cloning)
    if epoch % densification_interval == 0 and epoch > 0:

      # Calculate the norm of gradients
      gradient_norms = torch.norm(W.grad[persistent_mask][:, 7:9], dim=1, p=2) # Positional gradients
      gaussian_norms = torch.norm(torch.sigmoid(W.data[persistent_mask][:, 0:2]), dim=1, p=2) # Scale norms

      sorted_grads, sorted_grads_indices = torch.sort(gradient_norms, descending=True)
      sorted_gauss, sorted_gauss_indices = torch.sort(gaussian_norms, descending=True)

      large_gradient_mask = (sorted_grads > grad_threshold)
      large_gradient_indices = sorted_grads_indices[large_gradient_mask]

      large_gauss_mask = (sorted_gauss > gauss_threshold)
      large_gauss_indices = sorted_gauss_indices[large_gauss_mask]

      common_indices_mask = torch.isin(large_gradient_indices, large_gauss_indices)
      common_indices = large_gradient_indices[common_indices_mask]
      distinct_indices = large_gradient_indices[~common_indices_mask] # Those with large gradient but small scale

      # Split points with large coordinate gradient and large gaussian values and descale their gaussian
      if len(common_indices) > 0:
        print(f"Number of splitted points: {len(common_indices)}")
        start_index = current_marker + 1
        end_index = current_marker + 1 + len(common_indices)
        persistent_mask[start_index: end_index] = True
        W.data[start_index:end_index, :] = W.data[common_indices, :]
        scale_reduction_factor = 1.6
        W.data[start_index:end_index, 0:2] /= scale_reduction_factor
        W.data[common_indices, 0:2] /= scale_reduction_factor
        current_marker = current_marker + len(common_indices)

      # Clone points with large coordinate gradient but small gaussian values
      if len(distinct_indices) > 0:

        print(f"Number of cloned points: {len(distinct_indices)}")
        start_index = current_marker + 1
        end_index = current_marker + 1 + len(distinct_indices)
        persistent_mask[start_index: end_index] = True
        W.data[start_index:end_index, :] = W.data[distinct_indices, :]

        # Calculate the movement direction based on the positional gradient
        positional_gradients = W.grad[distinct_indices, 7:9]
        gradient_magnitudes = torch.norm(positional_gradients, dim=1, keepdim=True)
        normalized_gradients = positional_gradients / (gradient_magnitudes + 1e-8)

        step_size = 0.01

        # Move the cloned Gaussians slightly in gradient direction
        W.data[start_index:end_index, 7:9] += step_size * normalized_gradients

        current_marker = current_marker + len(distinct_indices)

    optimizer.step()
    scheduler.step()

    loss_history.append(loss.item())

    # --- Original Visualization Step ---
    # This visualization shows the output of the 'g_tensor_batch' (trained model output)
    # and the Ground Truth. It does not perform additional filter comparisons here.
    if epoch % display_interval == 0:
        num_subplots = 3 if display_loss else 2 # Default to 2 if loss is not displayed
        fig_size_width = 18 if display_loss else 12 # Default to 12 if loss is not displayed

        # Create subplots for generated image, ground truth, and optionally loss
        fig, ax = plt.subplots(1, num_subplots, figsize=(fig_size_width, 6))

        # Convert generated tensor to numpy for plotting
        generated_array = g_tensor_batch.cpu().detach().numpy()

        # Plot generated image (main output of the training loop)
        ax[0].imshow(generated_array)
        ax[0].set_title('2D Gaussian Splatting (Trained Baseline)') # Updated title
        ax[0].axis('off')

        # Plot Ground Truth image
        ax[1].imshow(target_tensor.cpu().numpy())
        ax[1].set_title('Ground Truth')
        ax[1].axis('off')

        # Optionally plot Loss vs Epochs
        if display_loss:
          ax[2].plot(range(epoch + 1), loss_history[:epoch + 1])
          ax[2].set_title('Loss vs. Epochs (Trained with Baseline)') # Updated title
          ax[2].set_xlabel('Epoch')
          ax[2].set_ylabel('Loss')
          ax[2].set_xlim(0, num_epochs)

        # Adjust layout and show plot
        plt.subplots_adjust(wspace=0.1)
        plt.pause(0.1) # Short pause to update plot in Colab

        # Save the generated image
        img = Image.fromarray((generated_array * 255).astype(np.uint8))
        filename = f"{epoch}.jpg"
        file_path = os.path.join(directory, filename)
        img.save(file_path)
        # fig.savefig(file_path, bbox_inches='tight') # Uncomment to save the matplotlib figure directly

        # Clear and close plot to free up memory
        plt.clf()
        plt.close()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, on {len(output)} points")

In [ ]:
# [Filter Comparison After Training]

import matplotlib.pyplot as plt
import torch
import numpy as np # Ensure numpy is imported if not already global

print("--- Generating Comparison Renders from the Trained Model ---")

# Ensure global variables from previous cells are accessible:
# KERNEL_SIZE, image_size, device
# W (the trained Gaussian parameters after the main training loop)
# persistent_mask (to select active Gaussians from W)
# target_tensor (the original ground truth image)
# generate_2D_gaussian_splatting function (defined in a previous cell)

# 1. Extract final active Gaussian parameters from the trained model (W)
final_output = W[persistent_mask]

final_batch_size = final_output.shape[0]

# Decode parameters using the same logic as in the training loop
final_scale = torch.sigmoid(final_output[:, 0:2])
final_rotation = np.pi / 2 * torch.tanh(final_output[:, 2])
final_alpha = torch.sigmoid(final_output[:, 3])
final_colours = torch.sigmoid(final_output[:, 4:7])
final_pixel_coords = torch.tanh(final_output[:, 7:9])

final_colours_with_alpha = final_colours * final_alpha.view(final_batch_size, 1)

# 2. Render images with different filter types using the FINAL trained parameters
#    debug_print=True is enabled to show numerical changes in covariance for the smallest Gaussian.

# A. Baseline Render (No Filter)
print("Rendering Baseline (No Filter) from trained model...")
compare_baseline_render = generate_2D_gaussian_splatting(
    KERNEL_SIZE, final_scale, final_rotation, final_pixel_coords, final_colours_with_alpha,
    image_size, device=device,
    filter_type=None,       # Apply no filter
    debug_print=True        # Enable debug prints for this comparison
)

# B. Low-Pass Filter Render
print("Rendering with Low-Pass Filter from trained model...")
compare_low_pass_render = generate_2D_gaussian_splatting(
    KERNEL_SIZE, final_scale, final_rotation, final_pixel_coords, final_colours_with_alpha,
    image_size, device=device,
    filter_type='low_pass', # Apply Low-Pass filter
    debug_print=True
)

# C. EWA Filter Render
print("Rendering with EWA Filter from trained model...")
compare_ewa_render = generate_2D_gaussian_splatting(
    KERNEL_SIZE, final_scale, final_rotation, final_pixel_coords, final_colours_with_alpha,
    image_size, device=device,
    filter_type='ewa',      # Apply EWA filter
    debug_print=True
)

# 3. Visualize the comparison
fig_compare, axes_compare = plt.subplots(1, 4, figsize=(20, 6))

# Plot Baseline Render
axes_compare[0].imshow(compare_baseline_render.detach().cpu().numpy())
axes_compare[0].set_title('Final Render:\nBaseline (Trained Model)')
axes_compare[0].axis('off')

# Plot Low-Pass Filtered Render
axes_compare[1].imshow(compare_low_pass_render.detach().cpu().numpy())
axes_compare[1].set_title('Final Render:\nLow-Pass Filter')
axes_compare[1].axis('off')

# Plot EWA Filtered Render
axes_compare[2].imshow(compare_ewa_render.detach().cpu().numpy())
axes_compare[2].set_title('Final Render:\nEWA Filter')
axes_compare[2].axis('off')

# Plot Ground Truth
axes_compare[3].imshow(target_tensor.cpu().numpy())
axes_compare[3].set_title('Ground Truth')
axes_compare[3].axis('off')

plt.suptitle("Final Model Output: Anti-Aliasing Filter Comparison", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("\n--- Comparison Rendering Complete ---")

# Discussion
In this assignment, you've observed the rendering process and result with baseline, baseline + Low-pass filter, baseline + EWA filter and ground truth.


Q1. Describe the specific aliasing artifacts you observed in the Baseline renderings (e.g., jagged edges, flickering, pixelation), particularly around smaller or rapidly changing Gaussians.

A1. [Answer here]

Q2. Visually compare how the Low-Pass and EWA filters modified these artifacts. What were the key visual differences between the Low-Pass and EWA filtered outputs, especially in terms of smoothness, detail preservation, and the overall appearance of both large and small Gaussians?

A2. [Answer here]

In [ ]:
drive.mount('/content/drive')
notebook_name = 'BaseCode_2DGS.ipynb'
!cp "/content/drive/MyDrive/Colab Notebooks/{notebook_name}" "/content/temp_notebook.ipynb"

output_dir = '/content/gdrive/MyDrive/Colab Notebooks/'
output_html_name = 'MyNotebook_output.html'

!jupyter nbconvert --to html "/content/temp_notebook.ipynb" --output-dir="{output_dir}" --output="{output_html_name}"

print(f"HTML file saved to: {output_dir}{output_html_name}")